### Состав команды:

- Поляков Александр (Aleksandr Polyakov)
- Востриков Алексей (Alexey Vostrikoff)
- Садыков Динар (Dinar_Sadykow)
- Федотов Андрей (Andrei Fedotov)
- Каштанкина Ксения
- Овчинников Алексей (Aleksey)
- Пьянков Алексей
- Лукошко Роберт (Robert)
- Поперечный Богдан (Bogdan2105)

### Описание данных

* train.csv - данные для обучения
* test.csv - данные для подготовки самбита и проверки
* sampleSubmission.csv - пример корректного но бесполезного сабмита
* other.csv - необязательные данные для доп.статистик и прочих извращений (например обучение word2vec-а)

### Описание полей

* id - внутренний идетификатор
* name - название вакансии
* description - текст вакансии
* target - класс заинтересованности

In [1]:
# Устанавливаем библиотеку для word2vec
!pip install gensim

In [2]:
!pip install cython

# Шаг 1. Загрузим файлы other.csv, train.cvs и test.csv
Файл other.csv содержит дополнительные неразмеченные данные, которые можно использовать для анализа.

In [1]:
%%time
import pandas as pd
other_df = pd.read_csv('other.csv','\t')
print('Я загрузила {} неразмеченных описания для обучения word2vec'.format(other_df['description'].size))
train_df = pd.read_csv('train.csv','\t')
print('Я загрузила {} размеченных тренировочных описаний'.format(train_df['description'].size))
test_df = pd.read_csv('test.csv','\t')
print('Я загрузила {} неразмеченных тестовых описаний'.format(test_df['description'].size))


Я загрузила 594534 неразмеченных описания для обучения word2vec
Я загрузила 200000 размеченных тренировочных описаний
Я загрузила 170179 неразмеченных тестовых описаний
CPU times: user 17.3 s, sys: 1.8 s, total: 19.1 s
Wall time: 19.2 s


# Шаг 2. Запишем функцию перекомпоновки описаний в лист слов

In [2]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def vac_to_wordlist(description, remove_stopwords = False):
    """
    берёт описание и превращает его в набор слов
    есть триггер включения выключения учета стоп-слов
    """
    #превращаем дескрипшен из хтмл в текст
    vac_text = BeautifulSoup(description, 'html5lib').get_text()
    
    #удаляем всё что не буквы
    vac_text = re.sub('[^а-яА-Яa-zA-Z]', ' ', vac_text)
    
    #превращаем текст в слова и переводим в нижний регистр
    words = vac_text.lower().split() 
    
    #опционально удаляем минус-слова
    if remove_stopwords:
        stops = set(stopwords.words('russian'))
        words = [w for w in words if not w in stops]
    return words

In [3]:
clean_vac = vac_to_wordlist( train_df['description'][0] , remove_stopwords = False)
clean_vac

['в',
 'новый',
 'магазин',
 'york',
 'хозтовары',
 'приглашаем',
 'на',
 'постоянную',
 'работу',
 'руководителя',
 'секции',
 'хозтовары',
 'обязанности',
 'организация',
 'эффективного',
 'и',
 'культурного',
 'обслуживания',
 'покупателей',
 'организация',
 'приемки',
 'сдачи',
 'товаров',
 'на',
 'склад',
 'консультация',
 'покупателей',
 'по',
 'вопросам',
 'касающимся',
 'оказываемых',
 'услуг',
 'контроль',
 'своевременной',
 'подачи',
 'товаров',
 'в',
 'торговую',
 'секцию',
 'проверка',
 'качества',
 'сроков',
 'годности',
 'тмц',
 'наличие',
 'маркировок',
 'ценников',
 'на',
 'товарах',
 'организация',
 'бесперебойной',
 'работы',
 'товарной',
 'секции',
 'и',
 'участка',
 'склада',
 'закрепленного',
 'за',
 'ней',
 'контроль',
 'за',
 'сохранностью',
 'товаров',
 'торгового',
 'оборудования',
 'и',
 'прочих',
 'материальных',
 'ценностей',
 'инвентаризация',
 'товаров',
 'требования',
 'знание',
 'товарной',
 'группы',
 'знание',
 'правил',
 'приемки',
 'товара',
 'опыт',

# Шаг 3. Пишем функцию, которая будет разделять описания на предложения

Word2Vec хочет работать с отдельными предложениями, а не с целыми описаниями

In [4]:
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [5]:
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
def description_to_sentences( description, tokenizer, remove_stopwords=False ):
    """
    Функция, которая разделяет текст на предложения
    """
    
    all_sentences = tokenizer.tokenize(description.strip())
    
    # проходим в цикле по всем предложениям
    sentences = []
    for sentence in all_sentences:
        # Если предложение пустое, то пропускаем его
        if len(sentence) > 0:
            # Иначе длбавляем описание в лист разделённое пословно
            sentences.append( vac_to_wordlist( sentence, remove_stopwords ))
    return sentences

In [6]:
clean_vac = description_to_sentences( train_df['description'][0] , tokenizer, remove_stopwords = False)
clean_vac

[['в',
  'новый',
  'магазин',
  'york',
  'хозтовары',
  'приглашаем',
  'на',
  'постоянную',
  'работу',
  'руководителя',
  'секции',
  'хозтовары',
  'обязанности',
  'организация',
  'эффективного',
  'и',
  'культурного',
  'обслуживания',
  'покупателей',
  'организация',
  'приемки',
  'сдачи',
  'товаров',
  'на',
  'склад',
  'консультация',
  'покупателей',
  'по',
  'вопросам',
  'касающимся',
  'оказываемых',
  'услуг',
  'контроль',
  'своевременной',
  'подачи',
  'товаров',
  'в',
  'торговую',
  'секцию',
  'проверка',
  'качества',
  'сроков',
  'годности',
  'тмц',
  'наличие',
  'маркировок',
  'ценников',
  'на',
  'товарах',
  'организация',
  'бесперебойной',
  'работы',
  'товарной',
  'секции',
  'и',
  'участка',
  'склада',
  'закрепленного',
  'за',
  'ней',
  'контроль',
  'за',
  'сохранностью',
  'товаров',
  'торгового',
  'оборудования',
  'и',
  'прочих',
  'материальных',
  'ценностей',
  'инвентаризация',
  'товаров',
  'требования',
  'знание',
  '

# Шаг 4. Соберём огромный "мешок предложений" из датасетов train и other

In [7]:
%%time
sentences = []
print('Собираю отдельные предложения из тренировочного набора: {} описаний'.format(train_df['description'].size))
i = 0
for description in train_df['description']:
    i += 1
    if (i % 10000) == 0:
        print('Прошла {} из {} описаний ({:.1%})'.format(i, train_df['description'].size,i/train_df['description'].size))
    sentences += description_to_sentences(description, tokenizer) # плюс равно чтобы добавил все листы, а не только первый
print('Собираю отдельные предложения из остального неразмеченного набора: {} описаний'.format(other_df['description'].size))
i = 0
for description in other_df['description']:
    i += 1
    if (i % 10000) == 0:
        print('Прошла {} из {} описаний ({:.1%})'.format(i, other_df['description'].size,i/other_df['description'].size))
    sentences += description_to_sentences(description, tokenizer)

Собираю отдельные предложения из тренировочного набора: 200000 описаний


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Прошла 2500 из 200000 описаний (1.2%)
Прошла 5000 из 200000 описаний (2.5%)
Прошла 7500 из 200000 описаний (3.8%)
Прошла 10000 из 200000 описаний (5.0%)
Прошла 12500 из 200000 описаний (6.2%)
Прошла 15000 из 200000 описаний (7.5%)
Прошла 17500 из 200000 описаний (8.8%)
Прошла 20000 из 200000 описаний (10.0%)
Прошла 22500 из 200000 описаний (11.2%)
Прошла 25000 из 200000 описаний (12.5%)
Прошла 27500 из 200000 описаний (13.8%)
Прошла 30000 из 200000 описаний (15.0%)
Прошла 32500 из 200000 описаний (16.2%)
Прошла 35000 из 200000 описаний (17.5%)
Прошла 37500 из 200000 описаний (18.8%)
Прошла 40000 из 200000 описаний (20.0%)
Прошла 42500 из 200000 описаний (21.2%)
Прошла 45000 из 200000 описаний (22.5%)
Прошла 47500 из 200000 описаний (23.8%)
Прошла 50000 из 200000 описаний (25.0%)
Прошла 52500 из 200000 описаний (26.2%)
Прошла 55000 из 200000 описаний (27.5%)
Прошла 57500 из 200000 описаний (28.7%)
Прошла 60000 из 200000 описаний (30.0%)
Прошла 62500 из 200000 описаний (31.2%)
Прошла 650

Прошла 305000 из 594534 описаний (51.3%)
Прошла 307500 из 594534 описаний (51.7%)
Прошла 310000 из 594534 описаний (52.1%)
Прошла 312500 из 594534 описаний (52.6%)
Прошла 315000 из 594534 описаний (53.0%)
Прошла 317500 из 594534 описаний (53.4%)
Прошла 320000 из 594534 описаний (53.8%)
Прошла 322500 из 594534 описаний (54.2%)
Прошла 325000 из 594534 описаний (54.7%)
Прошла 327500 из 594534 описаний (55.1%)
Прошла 330000 из 594534 описаний (55.5%)
Прошла 332500 из 594534 описаний (55.9%)
Прошла 335000 из 594534 описаний (56.3%)
Прошла 337500 из 594534 описаний (56.8%)
Прошла 340000 из 594534 описаний (57.2%)
Прошла 342500 из 594534 описаний (57.6%)
Прошла 345000 из 594534 описаний (58.0%)
Прошла 347500 из 594534 описаний (58.4%)
Прошла 350000 из 594534 описаний (58.9%)
Прошла 352500 из 594534 описаний (59.3%)
Прошла 355000 из 594534 описаний (59.7%)
Прошла 357500 из 594534 описаний (60.1%)
Прошла 360000 из 594534 описаний (60.6%)
Прошла 362500 из 594534 описаний (61.0%)
Прошла 365000 из

In [8]:
len(sentences)

1956571

In [10]:
sentences[1]

['заместителя',
 'заведующего',
 'секцией',
 'отделом',
 'ответственность',
 'лидерские',
 'качества',
 'опыт',
 'управления',
 'коллективом',
 'условия',
 'новый',
 'современный',
 'магазин',
 'розничной',
 'торговли',
 'хозтоварами',
 'хороший',
 'коллектив',
 'достойная',
 'зарплата',
 'премии',
 'соц',
 'пакет',
 'график',
 'работы',
 'пн',
 'пт',
 'с',
 'до',
 'плавающий',
 'место',
 'работы',
 'ул']

# Шаг 5. Тренируем модель

* [Документация 1](https://radimrehurek.com/gensim/models/word2vec.html)
* [Документация 2 (Google)](https://code.google.com/archive/p/word2vec/)


<br>


* **Architecture:** Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.
* **Training algorithm:** Hierarchical softmax (default) or negative sampling. For us, the default worked well.
* **Downsampling of frequent words:** The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.
* **Word vector dimensionality:** More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.
* **Context / window size:** How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).
* **Worker threads:** Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.
* **Minimum word count:** This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

In [12]:
%%time
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

#Укажем параметры

num_features = 300
min_word_count = 40
num_workers = 4
context = 10 
downsampling = 1e-3

from gensim.models import word2vec
print ('Обучаю word2vec')
model = word2vec.Word2Vec(sentences, workers=num_workers, 
            size=num_features, min_count = min_word_count, 
            window = context, sample = downsampling)
model.init_sims(replace=True)
model_name = '300features_40minwords_10context'
model.save(model_name)

2018-09-14 13:55:49,052 : INFO : 'pattern' package not found; tag filters are not available for English
2018-09-14 13:55:49,059 : INFO : collecting all words and their counts
2018-09-14 13:55:49,060 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-09-14 13:55:49,208 : INFO : PROGRESS: at sentence #10000, processed 543285 words, keeping 35863 word types


Обучаю word2vec


2018-09-14 13:55:49,367 : INFO : PROGRESS: at sentence #20000, processed 1092729 words, keeping 49798 word types
2018-09-14 13:55:49,517 : INFO : PROGRESS: at sentence #30000, processed 1632416 words, keeping 60370 word types
2018-09-14 13:55:49,676 : INFO : PROGRESS: at sentence #40000, processed 2175827 words, keeping 68796 word types
2018-09-14 13:55:49,825 : INFO : PROGRESS: at sentence #50000, processed 2717910 words, keeping 75913 word types
2018-09-14 13:55:49,978 : INFO : PROGRESS: at sentence #60000, processed 3270665 words, keeping 82466 word types
2018-09-14 13:55:50,141 : INFO : PROGRESS: at sentence #70000, processed 3800505 words, keeping 87805 word types
2018-09-14 13:55:50,302 : INFO : PROGRESS: at sentence #80000, processed 4355301 words, keeping 93091 word types
2018-09-14 13:55:50,461 : INFO : PROGRESS: at sentence #90000, processed 4908097 words, keeping 97849 word types
2018-09-14 13:55:50,628 : INFO : PROGRESS: at sentence #100000, processed 5453995 words, keeping

2018-09-14 13:56:00,918 : INFO : PROGRESS: at sentence #730000, processed 38313444 words, keeping 236783 word types
2018-09-14 13:56:01,043 : INFO : PROGRESS: at sentence #740000, processed 38739979 words, keeping 238176 word types
2018-09-14 13:56:01,187 : INFO : PROGRESS: at sentence #750000, processed 39197892 words, keeping 239715 word types
2018-09-14 13:56:01,335 : INFO : PROGRESS: at sentence #760000, processed 39635823 words, keeping 241534 word types
2018-09-14 13:56:01,489 : INFO : PROGRESS: at sentence #770000, processed 40103803 words, keeping 243000 word types
2018-09-14 13:56:01,636 : INFO : PROGRESS: at sentence #780000, processed 40556647 words, keeping 244661 word types
2018-09-14 13:56:01,787 : INFO : PROGRESS: at sentence #790000, processed 41042497 words, keeping 246362 word types
2018-09-14 13:56:01,935 : INFO : PROGRESS: at sentence #800000, processed 41507175 words, keeping 247848 word types
2018-09-14 13:56:02,073 : INFO : PROGRESS: at sentence #810000, processe

2018-09-14 13:56:11,628 : INFO : PROGRESS: at sentence #1440000, processed 72202738 words, keeping 315731 word types
2018-09-14 13:56:11,760 : INFO : PROGRESS: at sentence #1450000, processed 72649848 words, keeping 316373 word types
2018-09-14 13:56:11,899 : INFO : PROGRESS: at sentence #1460000, processed 73150819 words, keeping 317197 word types
2018-09-14 13:56:12,040 : INFO : PROGRESS: at sentence #1470000, processed 73633407 words, keeping 317907 word types
2018-09-14 13:56:12,186 : INFO : PROGRESS: at sentence #1480000, processed 74106524 words, keeping 318698 word types
2018-09-14 13:56:12,333 : INFO : PROGRESS: at sentence #1490000, processed 74599316 words, keeping 319403 word types
2018-09-14 13:56:12,457 : INFO : PROGRESS: at sentence #1500000, processed 75011727 words, keeping 319994 word types
2018-09-14 13:56:12,599 : INFO : PROGRESS: at sentence #1510000, processed 75549239 words, keeping 320749 word types
2018-09-14 13:56:12,721 : INFO : PROGRESS: at sentence #1520000,

...

2018-09-14 13:58:53,466 : INFO : EPOCH 2 - PROGRESS: at 95.00% examples, 955571 words/s, in_qsize 7, out_qsize 0
2018-09-14 13:58:54,467 : INFO : EPOCH 2 - PROGRESS: at 96.32% examples, 955945 words/s, in_qsize 7, out_qsize 0
2018-09-14 13:58:55,471 : INFO : EPOCH 2 - PROGRESS: at 97.54% examples, 956348 words/s, in_qsize 7, out_qsize 0
2018-09-14 13:58:56,477 : INFO : EPOCH 2 - PROGRESS: at 98.70% examples, 956874 words/s, in_qsize 7, out_qsize 0
2018-09-14 13:58:57,482 : INFO : EPOCH 2 - PROGRESS: at 99.94% examples, 957378 words/s, in_qsize 5, out_qsize 0
2018-09-14 13:58:57,506 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-14 13:58:57,512 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-14 13:58:57,520 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-14 13:58:57,521 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-14 13:58:57,523 : INFO : EPOCH - 2 : training on 96574840 raw w

2018-09-14 14:00:02,954 : INFO : EPOCH 3 - PROGRESS: at 74.16% examples, 942425 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:00:03,965 : INFO : EPOCH 3 - PROGRESS: at 75.47% examples, 943826 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:00:04,981 : INFO : EPOCH 3 - PROGRESS: at 76.84% examples, 945677 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:00:05,983 : INFO : EPOCH 3 - PROGRESS: at 78.26% examples, 947498 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:00:06,983 : INFO : EPOCH 3 - PROGRESS: at 79.65% examples, 949497 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:00:07,986 : INFO : EPOCH 3 - PROGRESS: at 81.08% examples, 951637 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:00:08,991 : INFO : EPOCH 3 - PROGRESS: at 82.49% examples, 952999 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:00:09,994 : INFO : EPOCH 3 - PROGRESS: at 83.79% examples, 954133 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:00:10,994 : INFO : EPOCH 3 - PROGRESS: at 85.05% examples, 955176 words/s, in_qsiz

2018-09-14 14:01:12,157 : INFO : EPOCH 4 - PROGRESS: at 64.34% examples, 1070211 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:01:13,160 : INFO : EPOCH 4 - PROGRESS: at 65.59% examples, 1069241 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:01:14,166 : INFO : EPOCH 4 - PROGRESS: at 67.00% examples, 1068685 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:01:15,178 : INFO : EPOCH 4 - PROGRESS: at 68.28% examples, 1067655 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:01:16,179 : INFO : EPOCH 4 - PROGRESS: at 69.66% examples, 1066869 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:01:17,181 : INFO : EPOCH 4 - PROGRESS: at 70.90% examples, 1065562 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:01:18,184 : INFO : EPOCH 4 - PROGRESS: at 72.11% examples, 1064943 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:01:19,187 : INFO : EPOCH 4 - PROGRESS: at 73.29% examples, 1064417 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:01:20,194 : INFO : EPOCH 4 - PROGRESS: at 74.66% examples, 1064390 words/s

2018-09-14 14:02:20,567 : INFO : EPOCH 5 - PROGRESS: at 52.27% examples, 1040703 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:02:21,573 : INFO : EPOCH 5 - PROGRESS: at 53.62% examples, 1042553 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:02:22,585 : INFO : EPOCH 5 - PROGRESS: at 54.99% examples, 1044201 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:02:23,585 : INFO : EPOCH 5 - PROGRESS: at 56.36% examples, 1045648 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:02:24,586 : INFO : EPOCH 5 - PROGRESS: at 57.73% examples, 1047239 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:02:25,585 : INFO : EPOCH 5 - PROGRESS: at 59.10% examples, 1048369 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:02:26,589 : INFO : EPOCH 5 - PROGRESS: at 60.50% examples, 1049724 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:02:27,589 : INFO : EPOCH 5 - PROGRESS: at 61.91% examples, 1051437 words/s, in_qsize 7, out_qsize 0
2018-09-14 14:02:28,602 : INFO : EPOCH 5 - PROGRESS: at 63.26% examples, 1052566 words/s

CPU times: user 26min 29s, sys: 10.7 s, total: 26min 40s
Wall time: 7min 7s


# Шаг 6. Играемся с word2vec

In [14]:
#наименее значащее слово из списка
model.wv.doesnt_match('ответственность лидерские качества'.split())

/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'качества'

In [15]:
model.wv.doesnt_match('офис крупная компания'.split())

/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'офис'

In [16]:
# понимает, что слово россия не город
model.wv.doesnt_match('москва россия калуга'.split())

/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'россия'

In [19]:
#наиболее близкие по смыслу слова
#бл%%ь, как она смогла????

model.most_similar('тула')

/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('киров', 0.7730791568756104),
 ('астрахань', 0.7520081400871277),
 ('рязань', 0.7477890253067017),
 ('саранск', 0.7462366223335266),
 ('смоленск', 0.7427210807800293),
 ('мончегорск', 0.7397128343582153),
 ('чебоксары', 0.7396755218505859),
 ('вологда', 0.7383654713630676),
 ('городец', 0.7333020567893982),
 ('чита', 0.7330015897750854)]

In [20]:
model.most_similar('английский')

/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('иностранный', 0.6687490940093994),
 ('китайский', 0.6018562316894531),
 ('польский', 0.5978714227676392),
 ('корейский', 0.5737009048461914),
 ('английским', 0.5712827444076538),
 ('англ', 0.5552904605865479),
 ('казахский', 0.5076484680175781),
 ('испанский', 0.5023046731948853),
 ('русский', 0.4914351999759674),
 ('немецкий', 0.4885626435279846)]

In [23]:
model.most_similar('хороший')

/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('отличный', 0.7360666990280151),
 ('прекрасный', 0.5923669934272766),
 ('высокий', 0.5381550788879395),
 ('дружеский', 0.5331491827964783),
 ('чудесный', 0.5301397442817688),
 ('душевный', 0.522240161895752),
 ('дружелюбный', 0.5203139781951904),
 ('базовый', 0.5194989442825317),
 ('сильный', 0.5179193615913391),
 ('приятный', 0.5139368176460266)]

In [25]:
model.most_similar('образование')

/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('образованиеопыт', 0.5119481086730957),
 ('образованиеусловия', 0.458920419216156),
 ('образования', 0.3992411196231842),
 ('аналогичный', 0.39759761095046997),
 ('предпочтительно', 0.3966981768608093),
 ('желательно', 0.39301443099975586),
 ('релевантный', 0.3916599750518799),
 ('микробиология', 0.3890746235847473),
 ('опыт', 0.3890298306941986),
 ('аспирантура', 0.3886414170265198)]

In [28]:
model.most_similar('настроение')

/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('зрение', 0.628744900226593),
 ('произношение', 0.5705733299255371),
 ('дарить', 0.5688424110412598),
 ('позитив', 0.4668731093406677),
 ('ориентирование', 0.4567759335041046),
 ('впечатление', 0.44994938373565674),
 ('воображение', 0.44943249225616455),
 ('начальство', 0.4452277421951294),
 ('понедельникам', 0.4411506950855255),
 ('юмора', 0.4282305836677551)]

In [29]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

2018-09-14 14:25:18,931 : INFO : loading Word2Vec object from 300features_40minwords_10context
2018-09-14 14:25:19,043 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2018-09-14 14:25:19,044 : INFO : loading vectors from 300features_40minwords_10context.wv.vectors.npy with mmap=None
2018-09-14 14:25:19,104 : INFO : setting ignored attribute vectors_norm to None
2018-09-14 14:25:19,105 : INFO : loading vocabulary recursively from 300features_40minwords_10context.vocabulary.* with mmap=None
2018-09-14 14:25:19,105 : INFO : loading trainables recursively from 300features_40minwords_10context.trainables.* with mmap=None
2018-09-14 14:25:19,106 : INFO : loading syn1neg from 300features_40minwords_10context.trainables.syn1neg.npy with mmap=None
2018-09-14 14:25:19,182 : INFO : setting ignored attribute cum_table to None
2018-09-14 14:25:19,182 : INFO : loaded 300features_40minwords_10context


In [30]:
model.most_similar('настроение')

/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2018-09-14 14:25:36,552 : INFO : precomputing L2-norms of word weight vectors
/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('зрение', 0.628744900226593),
 ('произношение', 0.5705733299255371),
 ('дарить', 0.5688424110412598),
 ('позитив', 0.4668731093406677),
 ('ориентирование', 0.4567759335041046),
 ('впечатление', 0.4499494433403015),
 ('воображение', 0.44943249225616455),
 ('начальство', 0.4452277421951294),
 ('понедельникам', 0.4411506950855255),
 ('юмора', 0.4282305836677551)]

In [35]:
type(model.syn0_lockf)

/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0_lockf` (Attribute will be removed in 4.0.0, use self.trainables.vectors_lockf instead).
  """Entry point for launching an IPython kernel.


numpy.ndarray

In [36]:
model.most_similar('россия')

/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('эстония', 0.6653217077255249),
 ('украина', 0.6576091051101685),
 ('бельгия', 0.6269100904464722),
 ('италия', 0.62611985206604),
 ('грузия', 0.6243079304695129),
 ('финляндия', 0.6174194812774658),
 ('белоруссия', 0.6120906472206116),
 ('азия', 0.6019402742385864),
 ('франция', 0.6013780832290649),
 ('испания', 0.598772406578064)]

In [39]:
model.syn0_lockf.shape

/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0_lockf` (Attribute will be removed in 4.0.0, use self.trainables.vectors_lockf instead).
  """Entry point for launching an IPython kernel.


(52725,)

In [42]:
model['россия'].shape

/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(300,)

# Шаг 7. Делаем для каждого описания свой вектор

In [61]:
import numpy as np

def makeFeatureVec (words, model, num_features):
    featureVec = np.zeros((num_features,), dtype='float32')
    nwords = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [62]:
def getAvgFeatureVecs(descriptions, model, num_features):
    counter = 0
    vacFeatureVecs = np.zeros((len(descriptions), num_features),dtype='float32')
    for description in descriptions:
        if counter%10000 == 0:
            print('Описание {} из {} обработано...'.format(counter, len(descriptions)))
        vacFeatureVecs[counter] = makeFeatureVec(description, model, num_features)
        counter = counter + 1
    return vacFeatureVecs

Вызываем функцию getAvgFeatureVecs для каждого описания

In [63]:
%%time
print('=========Делаем вектора для тренировочного датасета=========')
clean_train_vacs = []
i = 0
for vacancy in train_df['description']:
    if (i+1)%20000 == 0:
        print('Делаю набор слов для {} описания из {}'.format(i+1, train_df['description'].size))        
    clean_train_vacs.append(vac_to_wordlist(vacancy, remove_stopwords=True))
    i += 1

trainDataVecs = getAvgFeatureVecs(clean_train_vacs, model, num_features)
print('============Делаем вектора для тестового датасета============')
clean_test_vacs = []
i = 0
for vacancy in test_df['description']:
    if (i+1)%20000 == 0:
        print('Делаю набор слов для {} описания из {}'.format(i+1, test_df['description'].size))      
    clean_test_vacs.append(vac_to_wordlist(vacancy, remove_stopwords=True))
    i += 1    
testDataVecs = getAvgFeatureVecs(clean_test_vacs, model, num_features)

Делаем вектора для тренировочного датасета
Делаю 19999 из 200000
Делаю 39999 из 200000
Делаю 59999 из 200000
Делаю 79999 из 200000
Делаю 99999 из 200000
Делаю 119999 из 200000
Делаю 139999 из 200000
Делаю 159999 из 200000
Делаю 179999 из 200000
Делаю 199999 из 200000
Описание 0 из 200000 обработано...


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


Описание 10000 из 200000 обработано...
Описание 20000 из 200000 обработано...
Описание 30000 из 200000 обработано...


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


Описание 40000 из 200000 обработано...
Описание 50000 из 200000 обработано...
Описание 60000 из 200000 обработано...
Описание 70000 из 200000 обработано...
Описание 80000 из 200000 обработано...
Описание 90000 из 200000 обработано...
Описание 100000 из 200000 обработано...
Описание 110000 из 200000 обработано...
Описание 120000 из 200000 обработано...
Описание 130000 из 200000 обработано...
Описание 140000 из 200000 обработано...
Описание 150000 из 200000 обработано...
Описание 160000 из 200000 обработано...
Описание 170000 из 200000 обработано...
Описание 180000 из 200000 обработано...
Описание 190000 из 200000 обработано...
Делаем вектора для тестового датасета
Делаю 19999 из 170179
Делаю 39999 из 170179
Делаю 59999 из 170179
Делаю 79999 из 170179
Делаю 99999 из 170179
Делаю 119999 из 170179
Делаю 139999 из 170179
Делаю 159999 из 170179
Описание 0 из 170179 обработано...
Описание 10000 из 170179 обработано...
Описание 20000 из 170179 обработано...
Описание 30000 из 170179 обработано.

In [75]:
testDataVecs[40000:].shape

(130179, 300)

In [65]:
trainDataVecs

array([[ 0.01022952,  0.00219718, -0.00197375, ..., -0.0077006 ,
         0.00371153,  0.01921921],
       [ 0.02177511, -0.00804649, -0.00751581, ...,  0.01357641,
        -0.04320358,  0.02277128],
       [ 0.00184524,  0.00429932,  0.00345538, ..., -0.00247294,
        -0.01102962, -0.00936679],
       ...,
       [-0.00173076,  0.00233004,  0.01474884, ...,  0.01413206,
        -0.01448921,  0.00074065],
       [-0.00715773,  0.01531241,  0.00562211, ...,  0.00131948,
        -0.01723835,  0.01184722],
       [ 0.00730153, -0.0042074 ,  0.00426561, ..., -0.02178605,
        -0.01282327,  0.02013533]], dtype=float32)

# Шаг 8. Random Forest

In [84]:
%%time
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)
print ('Обучаем Random Forest на размеченных данных...')
# прибили все значения, которые NaN (сообщение об ошибке вылетало в прошлом пункте) и поставили вместо них нули
trainDataVecs[np.isnan(trainDataVecs)]=0 
forest = forest.fit(trainDataVecs, train_df['target'])
result = forest.predict(testDataVecs)
output = pd.DataFrame( data={'id':test_df['id'], 'target':result} )
output.to_csv( "Random_Forest_Word2Vec_Vector_Averaging.csv", sep=',', encoding='utf-8',index=False )

Обучаем Random Forest на размеченных данных...
CPU times: user 7min 58s, sys: 1.57 s, total: 7min 59s
Wall time: 7min 56s


In [85]:
#чисто из любопытства посмотреть
output.head()

,id,target
0,200000,1
1,200001,1
2,200002,1
3,200003,1
4,200004,0
